In [17]:
import os
import time
from riotwatcher import LolWatcher, ApiError

api_key = "RGAPI-a50ef4d9-fbd5-43e3-98db-4f27906915e7"
lol_watcher = LolWatcher(api_key)
my_region = 'NA1'

## Get PUUID of player
summoner_name = "duoking1"
me = lol_watcher.summoner.by_name(my_region,summoner_name)
puuid = me['puuid']
NA1 = "NA1"
ARAM = 450
RANKED_SOLO = 420

## Try to get 500 ARAM match IDs of player
matches = []
for big_loop_i in range(2):
    for small_loop_i in range(5):
        start = 500*big_loop_i+100*small_loop_i
        matchlist = lol_watcher.match.matchlist_by_puuid(puuid=puuid,region=NA1,start=start,queue=RANKED_SOLO,count=100)
        matches = matches + matchlist
    time.sleep(1.2)
print(len(matches))

with open('matches.txt', 'w') as filehandle:
    for match in matches:
        filehandle.write('%s\n' % match)

560


In [18]:
import pandas as pd

games_format = pd.DataFrame(columns=['match_id',
                        'ally_id_1','ally_id_2','ally_id_3','ally_id_4',
                        'enemy_id_1','enemy_id_2','enemy_id_3','enemy_id_4','enemy_id_5',
                        'my_champ','my_side',
                        'ally_champ_1','ally_champ_2','ally_champ_3','ally_champ_4',
                        'enemy_champ_1','enemy_champ_2','enemy_champ_3','enemy_champ_4','enemy_champ_5',
                        'win'])

In [19]:
games = []
for match_id in matches:
    match = lol_watcher.match.by_id(region=NA1,match_id=match_id)
    participants = match["metadata"]["participants"]
    infodto = match["info"]["participants"]
    ## Determine side of player (red or blue)
    if puuid in participants[:5]:
        blue = True
    else:
        blue = False

    ## Grabbing all of our teammates (not us!)
    ally_ids = ["","","",""]
    ally_champs = ["","","",""]
    offset = 0 if blue else 5
    ii = 0
    for t in range(5):
        curr = participants[offset+t]
        if curr != puuid:
            ally_ids[ii] = curr
            ally_champs[ii] = infodto[offset+t]["championName"]
            ii += 1
        else:
            ## write down our champ
            win = infodto[offset+t]["win"]
            my_champ = infodto[offset+t]["championName"]

    ## For legiility, initializing variables with easy names (can rewrite for performance)
    my_side = 'b' if blue else 'r'
    ally_id_1 = ally_ids[0] 
    ally_id_2 = ally_ids[1] 
    ally_id_3 = ally_ids[2] 
    ally_id_4 = ally_ids[3]
    enemy_id_1 = participants[5-offset]
    enemy_id_2 = participants[6-offset]
    enemy_id_3 = participants[7-offset]
    enemy_id_4 = participants[8-offset]
    enemy_id_5 = participants[9-offset]
    ally_champ_1 = ally_champs[0]
    ally_champ_2 = ally_champs[1]
    ally_champ_3 = ally_champs[2]
    ally_champ_4 = ally_champs[3]
    enemy_champ_1 = infodto[5-offset]["championName"]
    enemy_champ_2 = infodto[6-offset]["championName"]
    enemy_champ_3 = infodto[7-offset]["championName"]
    enemy_champ_4 = infodto[8-offset]["championName"]
    enemy_champ_5 = infodto[9-offset]["championName"]

    match_frame = pd.Series([match_id,
                            ally_id_1,ally_id_2,ally_id_3,ally_id_4,
                            enemy_id_1,enemy_id_2,enemy_id_3,enemy_id_4,enemy_id_5,
                            my_champ,my_side,
                            ally_champ_1,ally_champ_2,ally_champ_3,ally_champ_4,
                            enemy_champ_1,enemy_champ_2,enemy_champ_3,enemy_champ_4,enemy_champ_5,
                            win],
                            index=games_format.columns)
    # games = games.append(match_frame,ignore_index=True)
    # games = pd.concat([games, match_frame],axis=0)
    games.append(match_frame)
    time.sleep(1.2)
## Convert list to pandas DF
games = pd.DataFrame(games)

In [20]:
games_newest = games.head(230)
games_newest

,match_id,ally_id_1,ally_id_2,ally_id_3,ally_id_4,enemy_id_1,enemy_id_2,enemy_id_3,enemy_id_4,enemy_id_5,...,ally_champ_1,ally_champ_2,ally_champ_3,ally_champ_4,enemy_champ_1,enemy_champ_2,enemy_champ_3,enemy_champ_4,enemy_champ_5,win
0,NA1_4661400560,JI2nhWcxM9l42vFsRM3dUH3YYjxbu7unCg5Kjf8gKvmTMl...,fHDUdXXWG_qA-o-OgCH9NARUYyaPIW_jQ82c01_CyyV_g8...,wGBSjGPXHUaMhhMUvu9XqhsGGWbPXiWpuVaXPt39mvIKYu...,jIIVdKK1KZ9sW-kbSyDIhpIs_pkUaVRYl_wh3JZHSqzq5-...,9ZZOxxNWUasPytrmRBje0N0BuvlaSKnirc9hFu4eNpG-B-...,GB7OoiejX_vj0QWV2lc9RomgaUfFIzz6qrrjWyzBFFy59F...,h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQO...,8ArQ3-r_kYL46f_m9sYHgy_rSn-Oa6fPldgT7BfpSMHgGy...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,...,Akali,Neeko,KSante,Zeri,Gnar,Khazix,Lissandra,Varus,Thresh,True
1,NA1_4661360306,G7GyRdw80KvXVnr_Bf5RYU0ZyjAiLi6gCoTYQ7l4EKSiws...,wGBSjGPXHUaMhhMUvu9XqhsGGWbPXiWpuVaXPt39mvIKYu...,h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQO...,hZBGO4IpGSZpKsqA2p5y_GccXMsOH9nLL0vkqAvEaGR54n...,TLgQTsUE_ZVGUMrvh-gH7Mi2Jb5rFYn1w0VgmwVpYMPqL7...,_02gZ6cSYfEXtVr-8MH1NPYcS3syLt-w2lREQoplWwzken...,JI2nhWcxM9l42vFsRM3dUH3YYjxbu7unCg5Kjf8gKvmTMl...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,SxBsgk7X7KTtKG_NXKoDU8BxzCWZhxw5NJMpGVbgmPj6ZG...,...,Camille,Qiyana,Gragas,Ezreal,Mordekaiser,Talon,Renekton,Jhin,Karma,True
2,NA1_4661306593,yyNwa3KkvESfXwdnqeOv5ZnIJh8WqlLWe5pkRjy4CInZK8...,NrwMFPIyJmj77iHov9xYM3DFgD9XwifLmYUT_6ETrn9HRz...,F6l5cYQrKW4d3tRU39xFix2EYwnNSGlMcD0I6v9fodlgop...,sFx_drVi29XqrEKB2LAx6UGvmiBoorDDx7vEDFlOxT9mm9...,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,QzQd503tOEC4CMjqBx2Nbk11wjHavI33NI351bpDUd2ABV...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,9pgovaD3JwFyxeLCvuArfDgSTo81YTz-W-lvku9TLiF917...,Yv5gxC2RlkDvjk-_cbzwqJGYvqdLv8pmqIJxvUowzCgkr4...,...,Gragas,Rengar,Kennen,Zeri,KSante,Belveth,Sylas,Kaisa,Milio,False
3,NA1_4661286369,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,VPS6aTCAHA6IeUYtWMQKg8nsaCNTIvzTrNHGNoh9DD81DX...,HkaKpkzUH3DhH7f6nCWE2Fb5fqYEYeyW0OqpY_cd-Z8eLH...,BqObVX97_pzkE82KdHgrwKqbLNGOHAgFSdX2RzU4wk4yv1...,Yv5gxC2RlkDvjk-_cbzwqJGYvqdLv8pmqIJxvUowzCgkr4...,fHDUdXXWG_qA-o-OgCH9NARUYyaPIW_jQ82c01_CyyV_g8...,u0IkfCjrZ1RT83r6ccGmgtPW9GLyPzMhAB0KgnzI39V-hJ...,ocF0PJWJ9C6DNWraueJFhyhix7201uZFAlixC3iFL-X21O...,UH8DQY2XOjfe8zu9Yr4agCq9xP5HVaVTHO_BQNHwA1r-Nm...,...,Camille,Amumu,Karma,Tristana,Ornn,Khazix,Ahri,Jhin,Nautilus,True
4,NA1_4661243940,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gy...,uBYI9L8ggK1APLaIj-rlYOUhUNBPc7dNkkAXBnBrFw_Xuz...,v9n7P4UupQ2AHWho4ey8yNg7JLTg3kbJYmRM5I3xqRGnHb...,t1x0LSWtJ-DbCk1w87nPtcgOZ0-YvdQ1P6JJjWBKRPUDyP...,JNwe09JoTXd4Z_IXfv5hiice2ALXQvOhiomGKLPrPvUEZH...,Jn4x22UaPWsJOIS8eCU-Ab3mGohrDWtBFd8Xz45p5YRBy5...,-OB3aNmoRedM9pu6yatd2EftQp1gQVMN6NDs_0aVN1E3hH...,pU8ZFl9WJ01oMdmYJ6X1k0dtGyxIz7Hvmem5YPdXJ5na8c...,...,Tristana,Volibear,Cassiopeia,MissFortune,Jax,FiddleSticks,Renekton,Ezreal,Rakan,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,NA1_4544626145,n6XoaHs26BBLYBM0YLV4nV3ioxMxKCzz3L985q6m5Bwi0u...,S3GsSMbcNyG41SM0TZlpnhcUrtaB2p6Ib0IU_ihlYAVswg...,HYoGb9Y9jhLecjdid5fhZy3VAlFdMpBRa_UyURO3UPiFrc...,IAEubdjxdAHLYQVahMq2xSOKmHFg0tgZRDvKHteU0YtZhC...,x2d_rOl4fGofX6aK05NfJNxS36oYAVnitNNWLiwVPwC_zb...,eyZ3jXwwVSYRkpYLH7ao86VQZK9ZxDrOuhGhRmhjsB_pHa...,tzwLnjxEhznC0uBwLu7e8JzuIIsVlnbCWYoIzTosvMjhfz...,mwC3sNuwCLJFRmJ58287VbSedhfnCQb2bg4N4CpxPFmuaK...,n-Wq7hnWveV8YIrJfbidlA8-Q4_JR0_hDPFBhs9NdsGCej...,...,Yone,Diana,Shaco,Zac,Poppy,Kindred,Azir,Twitch,Heimerdinger,False
226,NA1_4544610899,9087vu1hZKALcasYplQZEkTWSg_xtnwFuvTg6WrG6vg5lm...,HkaKpkzUH3DhH7f6nCWE2Fb5fqYEYeyW0OqpY_cd-Z8eLH...,1w11cR1rvCXpeqGJa2-T4MFRCMic0jSIcVkFvEptf3QiJ3...,ffPW7uavSdI_6qM6yIQFOgIZxcb17ZDd_n4e3Rs1Av--5u...,SHkXnc2jBArncGF1EIkXle6M4OLzXBRMh7RQNN_pj-ujIt...,eyZ3jXwwVSYRkpYLH7ao86VQZK9ZxDrOuhGhRmhjsB_pHa...,ozYNYewfWExhNIp1OajTHFSVaNoJRiZ8UxKuXGHEqsLgec...,bFOLg1oJ3uZkdNFyb5HZe-JB9kZge_URL_A6gWwY52JCzl...,kDbWs97neQ5H

In [65]:
games_newest.to_csv('games_fixed.csv')

In [60]:
CHAMP = 'Alistar'
ally_count = 0
for col in ['ally_champ_1','ally_champ_2','ally_champ_3','ally_champ_4']:
    ally_count += games_newest.loc[games_newest[col] == CHAMP].shape[0]
enemy_count = 0
for col in ['enemy_champ_1','enemy_champ_2','enemy_champ_3','enemy_champ_4','enemy_champ_5']:
    enemy_count += games_newest.loc[games_newest[col] == CHAMP].shape[0]
print("(Ally Count, Enemy Count) = " + str((ally_count,enemy_count)))

(Ally Count, Enemy Count) = (2, 10)


In [59]:
games = games_newest
games

,match_id,ally_id_1,ally_id_2,ally_id_3,ally_id_4,enemy_id_1,enemy_id_2,enemy_id_3,enemy_id_4,enemy_id_5,...,ally_champ_1,ally_champ_2,ally_champ_3,ally_champ_4,enemy_champ_1,enemy_champ_2,enemy_champ_3,enemy_champ_4,enemy_champ_5,win
0,NA1_4661400560,JI2nhWcxM9l42vFsRM3dUH3YYjxbu7unCg5Kjf8gKvmTMl...,fHDUdXXWG_qA-o-OgCH9NARUYyaPIW_jQ82c01_CyyV_g8...,wGBSjGPXHUaMhhMUvu9XqhsGGWbPXiWpuVaXPt39mvIKYu...,jIIVdKK1KZ9sW-kbSyDIhpIs_pkUaVRYl_wh3JZHSqzq5-...,9ZZOxxNWUasPytrmRBje0N0BuvlaSKnirc9hFu4eNpG-B-...,GB7OoiejX_vj0QWV2lc9RomgaUfFIzz6qrrjWyzBFFy59F...,h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQO...,8ArQ3-r_kYL46f_m9sYHgy_rSn-Oa6fPldgT7BfpSMHgGy...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,...,Akali,Neeko,KSante,Zeri,Gnar,Khazix,Lissandra,Varus,Thresh,True
1,NA1_4661360306,G7GyRdw80KvXVnr_Bf5RYU0ZyjAiLi6gCoTYQ7l4EKSiws...,wGBSjGPXHUaMhhMUvu9XqhsGGWbPXiWpuVaXPt39mvIKYu...,h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQO...,hZBGO4IpGSZpKsqA2p5y_GccXMsOH9nLL0vkqAvEaGR54n...,TLgQTsUE_ZVGUMrvh-gH7Mi2Jb5rFYn1w0VgmwVpYMPqL7...,_02gZ6cSYfEXtVr-8MH1NPYcS3syLt-w2lREQoplWwzken...,JI2nhWcxM9l42vFsRM3dUH3YYjxbu7unCg5Kjf8gKvmTMl...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,SxBsgk7X7KTtKG_NXKoDU8BxzCWZhxw5NJMpGVbgmPj6ZG...,...,Camille,Qiyana,Gragas,Ezreal,Mordekaiser,Talon,Renekton,Jhin,Karma,True
2,NA1_4661306593,yyNwa3KkvESfXwdnqeOv5ZnIJh8WqlLWe5pkRjy4CInZK8...,NrwMFPIyJmj77iHov9xYM3DFgD9XwifLmYUT_6ETrn9HRz...,F6l5cYQrKW4d3tRU39xFix2EYwnNSGlMcD0I6v9fodlgop...,sFx_drVi29XqrEKB2LAx6UGvmiBoorDDx7vEDFlOxT9mm9...,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,QzQd503tOEC4CMjqBx2Nbk11wjHavI33NI351bpDUd2ABV...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,9pgovaD3JwFyxeLCvuArfDgSTo81YTz-W-lvku9TLiF917...,Yv5gxC2RlkDvjk-_cbzwqJGYvqdLv8pmqIJxvUowzCgkr4...,...,Gragas,Rengar,Kennen,Zeri,KSante,Belveth,Sylas,Kaisa,Milio,False
3,NA1_4661286369,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,VPS6aTCAHA6IeUYtWMQKg8nsaCNTIvzTrNHGNoh9DD81DX...,HkaKpkzUH3DhH7f6nCWE2Fb5fqYEYeyW0OqpY_cd-Z8eLH...,BqObVX97_pzkE82KdHgrwKqbLNGOHAgFSdX2RzU4wk4yv1...,Yv5gxC2RlkDvjk-_cbzwqJGYvqdLv8pmqIJxvUowzCgkr4...,fHDUdXXWG_qA-o-OgCH9NARUYyaPIW_jQ82c01_CyyV_g8...,u0IkfCjrZ1RT83r6ccGmgtPW9GLyPzMhAB0KgnzI39V-hJ...,ocF0PJWJ9C6DNWraueJFhyhix7201uZFAlixC3iFL-X21O...,UH8DQY2XOjfe8zu9Yr4agCq9xP5HVaVTHO_BQNHwA1r-Nm...,...,Camille,Amumu,Karma,Tristana,Ornn,Khazix,Ahri,Jhin,Nautilus,True
4,NA1_4661243940,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gy...,uBYI9L8ggK1APLaIj-rlYOUhUNBPc7dNkkAXBnBrFw_Xuz...,v9n7P4UupQ2AHWho4ey8yNg7JLTg3kbJYmRM5I3xqRGnHb...,t1x0LSWtJ-DbCk1w87nPtcgOZ0-YvdQ1P6JJjWBKRPUDyP...,JNwe09JoTXd4Z_IXfv5hiice2ALXQvOhiomGKLPrPvUEZH...,Jn4x22UaPWsJOIS8eCU-Ab3mGohrDWtBFd8Xz45p5YRBy5...,-OB3aNmoRedM9pu6yatd2EftQp1gQVMN6NDs_0aVN1E3hH...,pU8ZFl9WJ01oMdmYJ6X1k0dtGyxIz7Hvmem5YPdXJ5na8c...,...,Tristana,Volibear,Cassiopeia,MissFortune,Jax,FiddleSticks,Renekton,Ezreal,Rakan,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,NA1_4544626145,n6XoaHs26BBLYBM0YLV4nV3ioxMxKCzz3L985q6m5Bwi0u...,S3GsSMbcNyG41SM0TZlpnhcUrtaB2p6Ib0IU_ihlYAVswg...,HYoGb9Y9jhLecjdid5fhZy3VAlFdMpBRa_UyURO3UPiFrc...,IAEubdjxdAHLYQVahMq2xSOKmHFg0tgZRDvKHteU0YtZhC...,x2d_rOl4fGofX6aK05NfJNxS36oYAVnitNNWLiwVPwC_zb...,eyZ3jXwwVSYRkpYLH7ao86VQZK9ZxDrOuhGhRmhjsB_pHa...,tzwLnjxEhznC0uBwLu7e8JzuIIsVlnbCWYoIzTosvMjhfz...,mwC3sNuwCLJFRmJ58287VbSedhfnCQb2bg4N4CpxPFmuaK...,n-Wq7hnWveV8YIrJfbidlA8-Q4_JR0_hDPFBhs9NdsGCej...,...,Yone,Diana,Shaco,Zac,Poppy,Kindred,Azir,Twitch,Heimerdinger,False
226,NA1_4544610899,9087vu1hZKALcasYplQZEkTWSg_xtnwFuvTg6WrG6vg5lm...,HkaKpkzUH3DhH7f6nCWE2Fb5fqYEYeyW0OqpY_cd-Z8eLH...,1w11cR1rvCXpeqGJa2-T4MFRCMic0jSIcVkFvEptf3QiJ3...,ffPW7uavSdI_6qM6yIQFOgIZxcb17ZDd_n4e3Rs1Av--5u...,SHkXnc2jBArncGF1EIkXle6M4OLzXBRMh7RQNN_pj-ujIt...,eyZ3jXwwVSYRkpYLH7ao86VQZK9ZxDrOuhGhRmhjsB_pHa...,ozYNYewfWExhNIp1OajTHFSVaNoJRiZ8UxKuXGHEqsLgec...,bFOLg1oJ3uZkdNFyb5HZe-JB9kZge_URL_A6gWwY52JCzl...,kDbWs97neQ5H

In [60]:
games_ally = games[['ally_champ_1','ally_champ_2','ally_champ_3','ally_champ_4','win']].copy()
games_ally["win"] = games_ally["win"].replace({True: 1, False: 0})
games_ally["count"] = 1
games_ally_new = []
for ii in range(4):
    ally_champ_i = 'ally_champ_' + str(ii+1)
    new = games_ally[[ally_champ_i,'win','count']]
    new.columns = ['champ','ally_win','ally_count']
    games_ally_new.append(new)
games_ally_new = pd.concat(games_ally_new,ignore_index=True)
ally_sums = games_ally_new.groupby(['champ']).sum()

In [61]:
games_enemy = games[['enemy_champ_1','enemy_champ_2','enemy_champ_3','enemy_champ_4','enemy_champ_5','win']].copy()
games_enemy["win"] = games_enemy["win"].replace({True: 0, False: 1})
games_enemy["count"] = 1
games_enemy_new = []
for ii in range(5):
    enemy_champ_i = 'enemy_champ_' + str(ii+1)
    new = games_enemy[[enemy_champ_i,'win','count']]
    new.columns = ['champ','enemy_win','enemy_count']
    games_enemy_new.append(new)
games_enemy_new = pd.concat(games_enemy_new,ignore_index=True)
enemy_sums = games_enemy_new.groupby(['champ']).sum()

In [63]:
champ_counts = pd.merge(ally_sums,enemy_sums,on='champ',how='outer').fillna(0)
champ_counts

,ally_win,ally_count,enemy_win,enemy_count
champ,,,,
Aatrox,3.0,3.0,3.0,6.0
Ahri,6.0,8.0,3.0,8.0
Akali,1.0,4.0,3.0,6.0
Akshan,5.0,9.0,5.0,7.0
Alistar,2.0,2.0,7.0,10.0
...,...,...,...,...
RekSai,0.0,0.0,1.0,4.0
Skarner,0.0,0.0,0.0,1.0
Sona,0.0,0.0,1.0,3.0


In [64]:
champ_counts["total_win"] = champ_counts["ally_win"] + champ_counts["enemy_win"]
champ_counts["total_count"] = champ_counts["ally_count"] + champ_counts["enemy_count"]
champ_counts["ally_win"] = champ_counts["ally_win"] / champ_counts["ally_count"]
champ_counts["enemy_win"] = champ_counts["enemy_win"] / champ_counts["enemy_count"]
champ_counts["total_win"] = champ_counts["total_win"] / champ_counts["total_count"]
champ_counts = champ_counts.fillna(0).sort_values(by='champ')
champ_counts.to_csv("champ_counts.csv")

In [57]:
champ_counts

,ally_win,ally_count,enemy_win,enemy_count,total_win,total_count
champ,,,,,,
Aatrox,0.111111,3.0,0.013889,6.0,0.046296,9.0
Ahri,0.011719,8.0,0.005859,8.0,0.008789,16.0
Akali,0.015625,4.0,0.013889,6.0,0.014583,10.0
Akshan,0.006859,9.0,0.014577,7.0,0.010236,16.0
Alistar,0.250000,2.0,0.007000,10.0,0.047500,12.0
...,...,...,...,...,...,...
Zeri,0.003005,11.0,0.002915,7.0,0.002970,18.0
Ziggs,0.250000,2.0,0.000000,5.0,0.071429,7.0
Zilean,1.000000,1.0,0.250000,2.0,0.500000,3.0


In [45]:
## Get ally ids
ids_ally = games[['ally_id_1','ally_id_2','ally_id_3','ally_id_4','win']].copy()
ids_ally["win"] = games_ally["win"].replace({True: 1, False: 0})
ids_ally["count"] = 1
ids_ally_new = []
for ii in range(4):
    ally_id_i = 'ally_id_' + str(ii+1)
    new = ids_ally[[ally_id_i,'win','count']]
    new.columns = ['id','ally_win','ally_count']
    ids_ally_new.append(new)
ids_ally_new = pd.concat(ids_ally_new,ignore_index=True)

## Get enemy ids
ids_enemy = games[['enemy_id_1','enemy_id_2','enemy_id_3','enemy_id_4','enemy_id_5','win']].copy()
ids_enemy["win"] = ids_enemy["win"].replace({True: 0, False: 1})
ids_enemy["count"] = 1
ids_enemy_new = []
for ii in range(5):
    enemy_id_i = 'enemy_id_' + str(ii+1)
    new = ids_enemy[[enemy_id_i,'win','count']]
    new.columns = ['id','enemy_win','enemy_count']
    ids_enemy_new.append(new)
ids_enemy_new = pd.concat(ids_enemy_new,ignore_index=True)

## Group sort and whateverf
ally_id_sums = ids_ally_new.groupby(['id']).sum()
enemy_id_sums = ids_enemy_new.groupby(['id']).sum()
id_sums = pd.merge(ally_id_sums,enemy_id_sums,on='id')
id_sums['total_win'] = id_sums['ally_win'] + id_sums['enemy_count'] - id_sums['enemy_win']
id_sums["total_count"] = id_sums['ally_count'] + id_sums['enemy_count']

In [46]:
id_sums_relevant = id_sums[id_sums.total_count >= 5].sort_values('total_count',ascending=False)
id_sums_relevant

,ally_win,ally_count,enemy_win,enemy_count,total_win,total_count
id,,,,,,
UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gyWKIc11ulF4niI-EwNQTA7O7BQqyrGudA,6,10,4,6,8,16
amzCw79znENELIUiVdGfqJTDe6jlJhFFxA03yjKchJ5yM4uGpMzyjokPgS_2JPLIAhZqADudzRWipA,2,5,2,6,6,11
h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQOGTgAGKOJkOUTHHaNFnhnwgm_zY7V6VHg,5,6,3,5,7,11
AKWVwS5AKoN3GlPEi8M9GSxr4D1qVeY_dwDM5hIp-xkm0gG1B9x4tC6UHkv5Q2402hqrEL0sfbyfXg,3,4,3,7,7,11
OuMgG87brAjyf1VfYmHXHYOIpIK-ocVCZ5wThvhQ1MyQBgWZO-K0mT6VM9LGHAYWyh6r8XDLPXWjtQ,1,4,4,6,3,10
...,...,...,...,...,...,...
X4ZWIgScPfCEkhciBH4lSG1AJgWGWQasEA14CPz2coqbHLYc0uEoSmDGoouhSeiUu0ImQ6mILirTGQ,1,2,1,3,3,5
YZ-HFZdnHSCemQdQs2y4JLGzvWLoxwNTOv8VXRl4qpbi-_aY9WlHO9HFU4iJT3G0_HYou7uCEzvyBA,1,2,1,3,3,5
bHrDikb01hYFfTwJQz0R4OSFXaPDxW2x2QJGduMMn9Qq2Bv1HDupJ2OpTBE5f0CcQEyP0dhbE3jKHg,2,2,1,3,4,5


In [47]:
import os
import time
from riotwatcher import LolWatcher, ApiError

api_key = "RGAPI-a50ef4d9-fbd5-43e3-98db-4f27906915e7"
lol_watcher = LolWatcher(api_key)
my_region = 'NA1'
summoners = []
for puuid in id_sums_relevant.index:
    dude = lol_watcher.summoner.by_puuid(encrypted_puuid=puuid,region="NA1")
    summoner_name = dude["name"]
    summoners.append(summoner_name)
    time.sleep(1.2)

In [48]:
id_sums_relevant["Summoner Name"] = summoners
id_sums_relevant["total_win"] = id_sums_relevant["ally_win"] + id_sums_relevant["enemy_win"]
id_sums_relevant["total_count"] = id_sums_relevant["ally_count"] + id_sums_relevant["enemy_count"]
id_sums_relevant["ally_win"] = id_sums_relevant["ally_win"] / id_sums_relevant["ally_count"]
id_sums_relevant["enemy_win"] = id_sums_relevant["enemy_win"] / id_sums_relevant["enemy_count"]
id_sums_relevant["total_win"] = id_sums_relevant["total_win"] / id_sums_relevant["total_count"]
id_sums_relevant.to_csv("id_sums_relevant.csv")

                                                    ally_win  ally_count   
id                                                                         
UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gyW...  0.600000          10  \
amzCw79znENELIUiVdGfqJTDe6jlJhFFxA03yjKchJ5yM4u...  0.400000           5   
h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQOG...  0.833333           6   
AKWVwS5AKoN3GlPEi8M9GSxr4D1qVeY_dwDM5hIp-xkm0gG...  0.750000           4   
OuMgG87brAjyf1VfYmHXHYOIpIK-ocVCZ5wThvhQ1MyQBgW...  0.250000           4   
...                                                      ...         ...   
X4ZWIgScPfCEkhciBH4lSG1AJgWGWQasEA14CPz2coqbHLY...  0.500000           2   
YZ-HFZdnHSCemQdQs2y4JLGzvWLoxwNTOv8VXRl4qpbi-_a...  0.500000           2   
bHrDikb01hYFfTwJQz0R4OSFXaPDxW2x2QJGduMMn9Qq2Bv...  1.000000           2   
dcrcf9eNpNL9pQPDycXOkgdkbHN8TgaYjcSwhEJoQBbu0tc...  1.000000           2   
zoP2VHuBwAs7jDp7NFnQjq761qssqJBlvAPOIZ-gTto1TqZ...  0.333333           3   

           

In [24]:
import os
import time
from riotwatcher import LolWatcher, ApiError

api_key = "RGAPI-a50ef4d9-fbd5-43e3-98db-4f27906915e7"
lol_watcher = LolWatcher(api_key)
my_region = 'NA1'


In [27]:
puuid = "UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gyWKIc11ulF4niI-EwNQTA7O7BQqyrGudA"
dude = lol_watcher.summoner.by_puuid(encrypted_puuid=puuid,region="NA1")

In [35]:
dude["name"]

'soloking1'

In [60]:
enemy_ids_sums = ids_enemy_new.groupby(['id']).sum().sort_values('enemy_count',ascending=False)

In [63]:
enemy_ids_sums[enemy_ids_sums.enemy_count >= 5]

,enemy_win,enemy_count
id,,
wap1DpTaJDZ2PnRQRMQBwJcp5-fjGq3HFc-0m6sB6qz55w2Zcq9Gba_zXYDmPRJbCwn3kSc469LziA,3,7
onZP0tygdtfiop8b4WvFubrBGpeBaxV54eKJ6HJmGHE-zVoMBFoVeGDporWiXKDlBmbOe4xE9VL9AQ,5,7
OuMgG87brAjyf1VfYmHXHYOIpIK-ocVCZ5wThvhQ1MyQBgWZO-K0mT6VM9LGHAYWyh6r8XDLPXWjtQ,2,6
MBaAn00hSXfmU9wsN_zGI0yNRy791BKc80U-b5c04_438j4J3q5pBOJKOZcqbqM7DnHv1H9UnHdmew,4,6
eVXs22zSxLFOaClYDfA8C2Cn640jbbv1wqdf9KcfYwYJ1Vl2qJcY_QTmGujz8YOEB4lvSFSGNeAgVA,5,6
amzCw79znENELIUiVdGfqJTDe6jlJhFFxA03yjKchJ5yM4uGpMzyjokPgS_2JPLIAhZqADudzRWipA,4,6
4xSQF-tFIYK_YXM9wkR1QRwmhNO-bcdByCfHIkXSN6aBBMMiWDJtUoOf7_wPd4Q_m41Ukw5Bbbz7WA,3,6
AKWVwS5AKoN3GlPEi8M9GSxr4D1qVeY_dwDM5hIp-xkm0gG1B9x4tC6UHkv5Q2402hqrEL0sfbyfXg,4,6
pSzPod5MXiqzw8L-arE9hdEFeeDkYvOnIFXklHVAxznmEuLfaPpTkujZf_GD22f6NewmEOCf9drkPw,0,5
